In [ ]:
import param
import panel as pn
pn.extension()
pn.config.sizing_mode="stretch_width"
from IPython.display import display, HTML
js = """
<script src="https://unpkg.com/@webcomponents/webcomponentsjs@2.0.0/webcomponents-loader.js"></script>
<script type="module" src="https://unpkg.com/wired-elements@0.6.4/dist/wired-elements.bundled.js"></script>
"""
display(HTML(js))
class CheckBox(pn.pane.HTML):
        tag = param.String("wired-radio", constant=True)
        js = param.List(
            ["https://unpkg.com/wired-elements@0.6.4/dist/wired-elements.bundled.js"], constant=True
        )

        attributes = param.List(["value", "checked"], constant=True)

        checked = param.Boolean(True)
        disabled = param.Boolean(False)
        
        def __init__(self, **params):
            super().__init__(**params)
            self._set_object()
        
        @param.depends("checked", watch=True)
        def _set_object(self):
            if self.checked:
                object = "<wired-radio checked>Radio Two</wired-radio>"
            else:
                object = "<wired-radio>Radio Two</wired-radio>"
            if self.object != object:
                self.object = object
            
        @param.depends("object", watch=True)
        def _set_checked(self):
            checked="checked" in self.object
            if self.checked != checked:
                self.checked = checked
        
        
checkbox = CheckBox()
pn.Column(pn.Param(checkbox, parameters=["object", "checked"]), checkbox)


In [ ]:
from panel.components.wired.buttons import RadioButton, CheckBox, Slider, ComboBox

js = """
<script src="https://unpkg.com/@webcomponents/webcomponentsjs@2.0.0/webcomponents-loader.js"></script>
<script type="module" src="https://unpkg.com/wired-elements@0.6.4/dist/wired-elements.bundled.js"></script>
"""

radio_button = RadioButton()
check_box = CheckBox()
slider = Slider()
combobox = ComboBox()
# video = Video(height=500)
pn.Column(
    pn.pane.HTML(js),
    radio_button, pn.Param(radio_button.param.html),
    check_box, pn.Param(check_box.param.html),
    slider, pn.Param(slider.param.html),
    combobox, pn.Param(combobox.param.html),
    # video, pn.Param(video.param.html),
    ).servable()